## Curso de Inteligencia Artificial

### Notas

* Entregar un cuaderno jupyter o colab con todas tus respuestas. No se aceptan otro tipo de formato.
* **En esta tarea se evaluará todas las preguntas y no se asignará puntaje a preguntas incompletas**.
* Responde las preguntas en orden.
* Incluir imágenes o formas de verificar la ejecución de los programas planteados. Explica toda tus respuestas.
* Puedes usar tus apuntes y consultar fuentes externas. Las únicas restricciones son que el trabajo sea completamente individual (no puedes comunicarte con tus compañeros ni con ninguna otra persona) y que cualquier material que tomes de otra fuente debe ser debidamente citado (de lo contrario será considerado plagio).

* **Tiempo estimado de resolución: 90 minutos**.

* Luego de cada pregunta se indica en comentarios las secciones donde debes incluir tu código. *No modifiques el código fuera de esas secciones*.

```
### Inicio de tu código
...Tu código
### Fin de tu código
```

En algunos lugares de la prueba hay algunas sentencias `assert` que emitirán un mensaje de error si tu solución tiene algún error particular. Cuando tu error sea correcto no emitirán ningún mensaje.

Usaremos una versión del conjunto de datos *Lending Club Data*.

LendingClub es una compañía de préstamos *peer-to-peer* que conecta directamente a los prestatarios con potenciales prestamistas/inversionistas.

Construirás un modelo de clasificación para predecir si un préstamo realizado a través del LendingClub tiene probabilidad de no ser pagado *(default)*.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

loans = pd.read_csv("lending-club-data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Identificar las columnas

**Pregunta 2.1 (0.5 puntos):** Carga en una variable de nombre `all_columns` el nombre de todas las columnas del conjunto de datos.

In [2]:
### Inicio de tu código
all_columns = loans.columns
### Fin de tu código
print('El nombre de todas las columnas es: \n', all_columns)

El nombre de todas las columnas es: 
 Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       '

### Preprocesar la columna etiqueta

**Pregunta 2.2 (1 punto):** La columna que contiene la información que queremos predecir se llama `bad_loans`. En esta columna, el valor 1 significa un préstamo riesgoso (malo), mientras que 0 significa un préstamos seguro.

Para hacer el trabajo más intuitivo, crea una nueva columna `safe_loans` con el siguiente valor:

* +1 si es un préstamo seguro
* -1 si es un préstamos riesgoso (malo)

In [15]:
print('Distribución de la columna bad_loans:')
print(loans['bad_loans'].value_counts())

### Inicio de tu código
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: 1 if x == 0 else -1)
### Fin de tu código

print('\nDistribución de la nueva columna safe_loans:')
print(loans['safe_loans'].value_counts())


Distribución de la columna bad_loans:
0    99457
1    23150
Name: bad_loans, dtype: int64

Distribución de la nueva columna safe_loans:
 1    99457
-1    23150
Name: safe_loans, dtype: int64


### Porcentaje de las clases a predecir

**Pregunta 2.3 (1 punto):** Calcula la distribución en **porcentaje** de préstamos malos y préstamos buenos (debe sumar 100%).

In [16]:
### Inicio de tu código
good, bad = loans['safe_loans'].value_counts()
print('La distribución en porcentaje de préstamos malos y préstamos buenos es:')
print(f"Malos = {(bad/(bad+good))*100} %")
print(f"Buenos = {(good/(bad+good))*100} %")
### Fin de tu código

La distribución en porcentaje de préstamos malos y préstamos buenos es:
Malos = 18.881466800427383 %
Buenos = 81.11853319957262 %


### Balanceo de datos con submuestreo

**Pregunta 2.4 (1 punto):** Una manera de abordar conjuntos de datos desbalanceados es con un submuestreo *(undersampling)* de la clase más grande hasta que la distribución de clases sea mitad y mitad. Vamos a realizar un submuestreo de los préstamos buenos para balancear nuestro conjunto de datos. Ello significa que vamos a descartar muchas observaciones. 

* Pon en una variable `risky_loans` todos y solo los préstamos malos.
* Pon en una variable `safe_loans` una muestra aleatoria de préstamos buenos **del mismo tamaño** que la cantidad de préstamos malos. (Usa [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) con el atributo `random_state=0`)
* Junta en una nueva variable `loans_balanced` los dos grupos anteriores: `risky_loans` y `safe_loans` (Puedes usar [pandas.concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html))

In [52]:
### Inicio de tu código
risky_loans = loans[lambda x: x['safe_loans'] == -1]
risky_loans = risky_loans['safe_loans']
safe_loans = loans['safe_loans'][loans['safe_loans'] == 1].sample(n=bad, random_state=0)
loans_balanced = pd.concat([risky_loans, safe_loans])
### Fin de tu código

assert len(loans_balanced) == 2 * len(risky_loans), 'loans_balanced quedar con el doble de observaciones que risky_loans'
print('\nDistribución de la columna safe_loans en loans_balanced:')
print(loans_balanced.value_counts())


Distribución de la columna safe_loans en loans_balanced:
-1    23150
 1    23150
Name: safe_loans, dtype: int64


### Selección de características

**Pregunta 2.5 (0.5 puntos):** Asigna a una variable `loans_subset` sólo el siguiente subconjunto de características que son las que usaremos:

```python
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
```

Asimismo, asigna a una variable **`y`** los valores de la columna `safe_loans`

In [71]:
### Inicio de tu código
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

idx = list(loans_balanced.index)

loans_subset = loans[features].iloc[idx]
y = loans_balanced.to_numpy()
### Fin de tu código

assert len(loans_subset) == len(loans_balanced), 'asegúrate de usar loans_balanced'
assert 'safe_loans' not in loans_subset.columns.values, 'safe_loans no debe estar en loans_subset'
assert len(loans_subset) == len(y), 'loans_subset y Y no tienen el mismo número de observaciones'

### Conversión de variables categóricas

**Pregunta 2.6 (0.5 puntos):** Usando [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) convierte las variables categóricas de `loans_subset` en variables numéricas *one-hot*. Guarda el nuevo conjunto de datos en `X`.

In [72]:
### Inicio de tu código
X = pd.get_dummies(loans_subset)
### Fin de tu código

assert len(X) == len(y), 'X y Y no tienen el mismo número de observaciones'
assert len(X.columns.values) == 67

### Separación del conjunto de datos

**Pregunta 2.7 (0.5 puntos):** Empleando `sklearn.model_selection.train_test_split` separa el conjunto de datos en un 90% para entrenamiento y validación (`X_trainval`, `y_trainval`), y 10% para pruebas (`X_test`, `y_test`).

Luego separa (`X_trainval`, `y_trainval`) en un 80% para entrenamiento (`X_train`, `y_train`) y 20% para validación (`X_val`, `y_val`)

In [74]:
from sklearn.model_selection import train_test_split

### Inicio de tu código
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.20, random_state=42)
### Fin de tu código

assert X_trainval.shape[0] == y_trainval.shape[0], 'Dimensiones incorrectas'
assert X_train.shape[0] == y_train.shape[0], 'Dimensiones incorrectas'
assert X_val.shape[0] == y_val.shape[0], 'Dimensiones incorrectas'
assert X_test.shape[0] == y_test.shape[0], 'Dimensiones incorrectas'

### Selección del mejor modelo

**Pregunta 2.8 (5 puntos):** Entrena un modelo SVM, un modelo de ensamble (RandomForests, AdaBoost u otro), y un tercer modelo de tu elección, con las siguientes indicaciones:

* Se calificará el uso apropiado de la normalización (Scaling) de datos
* Se calificará el uso apropiado de una técnica para la selección de los mejores parámetros de cada modelo (p.ej. búsqueda en grilla o búsqueda aleatoria)
* Reporta para cada modelo la exactitud (accuracy), precisión y exhaustividad (recall) **en el conjunto de pruebas.**
* Discute finalmente los resultados que hayas obtenido. (Min. 3 líneas).

In [75]:
## Normalización de datos

### Inicio de tu código
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
### Fin de tu código

In [77]:
## Modelo SVM: ajuste y selección de mejores parámetros, evaluación en el conjunto de pruebas

### Inicio de tu código
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

pipe = make_pipeline(StandardScaler(), svm.SVC(max_iter=10))
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)
### Fin de tu código

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


0.5069114470842333

In [81]:
## Modelo de ensamble: ajuste y selección de mejores parámetros, evaluación en el conjunto de pruebas

### Inicio de tu código
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=svm.SVC(max_iter=15), n_estimators=10, random_state=0).fit(X_train_scaled, y_train)
X_test_scaled = scaler.transform(X_test)
clf.score(X_test_scaled, y_test)
### Fin de tu código

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:

0.5041036717062635

In [ ]:
## Modelo 3: ajuste y selección de mejores parámetros, evaluación en el conjunto de pruebas

### Inicio de tu código
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'gamma': [1, 0.01],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(svm.SVC(), param_grid,refit=True,verbose=2)
grid.fit(X_train_scaled, y_train)
grid.score(X_test_scaled, y_test)

### Fin de tu código

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


### Discusión  de resultados


In [ ]:
## Respuestas